In [ ]:


import cv2
import numpy as np
import time

# ----------------- utils -----------------
def point_rect_dist(px, py, rx1, ry1, rx2, ry2):
    
    if px < rx1:
        dx = rx1 - px
    elif px > rx2:
        dx = px - rx2
    else:
        dx = 0
    if py < ry1:
        dy = ry1 - py
    elif py > ry2:
        dy = py - ry2
    else:
        dy = 0
    return np.hypot(dx, dy)

# ----------------- main -----------------
def main():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Cannot open webcam")
        return

    show_mask = False
    paused = False

    # FPS measurement
    fps = 0.0
    last_time = time.time()

    while True:
        if not paused:
            ret, frame = cap.read()
            if not ret:
                print("Failed read frame")
                break

            frame = cv2.flip(frame, 1)  # mirror
            h, w = frame.shape[:2]

            # Virtual object 
            rect_w = int(w * 0.28)
            rect_h = int(h * 0.35)
            rect_x1 = w // 2 - rect_w // 2
            rect_y1 = h // 2 - rect_h // 2
            rect_x2 = rect_x1 + rect_w
            rect_y2 = rect_y1 + rect_h

            #  Skin detection (YCrCb) 
            ycrcb = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)
            
            lower = np.array([0, 133, 77], dtype=np.uint8)
            upper = np.array([255, 173, 127], dtype=np.uint8)
            mask = cv2.inRange(ycrcb, lower, upper)

            
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
            mask = cv2.GaussianBlur(mask, (7,7), 0)

            #  Find contours 
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            hand_contour = None
            hand_bbox = (0,0,0,0)
            if contours:
                
                contours = sorted(contours, key=cv2.contourArea, reverse=True)
                
                for cnt in contours:
                    area = cv2.contourArea(cnt)
                    if area > (w*h)*0.002:  
                        hand_contour = cnt
                        x,y,ww,hh = cv2.boundingRect(cnt)
                        hand_bbox = (x,y,ww,hh)
                        break

            # default state
            state = "SAFE"
            state_color = (0,255,0)  # green

            min_dist = None
            if hand_contour is not None:
                
                cv2.drawContours(frame, [hand_contour], -1, (200,200,0), 2)
                x,y,ww,hh = hand_bbox
                cv2.rectangle(frame, (x,y), (x+ww, y+hh), (255,200,0), 1)

                
                contour_pts = hand_contour.reshape(-1,2)
                dists = [point_rect_dist(int(px), int(py), rect_x1, rect_y1, rect_x2, rect_y2) for (px,py) in contour_pts]
                min_dist = float(min(dists))

                
                hand_size = max(ww, hh, 1)
                
                danger_thresh = 0.25 * hand_size    
                warning_thresh = 0.9 * hand_size    
                
                if min_dist <= danger_thresh:
                    state = "DANGER"
                    state_color = (0,0,255)
                elif min_dist <= warning_thresh:
                    state = "WARNING"
                    state_color = (0,165,255)  
                else:
                    state = "SAFE"
                    state_color = (0,255,0)
            else:
                min_dist = None
                state = "SAFE"
                state_color = (0,255,0)

            #  Draw virtual object and overlays 
            
            thickness = 3
            cv2.rectangle(frame, (rect_x1, rect_y1), (rect_x2, rect_y2), state_color, thickness)

            
            if hand_contour is not None:
                
                idx = int(np.argmin(dists))
                px, py = contour_pts[idx]
                
                cx = min(max(px, rect_x1), rect_x2)
                cy = min(max(py, rect_y1), rect_y2)
                cv2.line(frame, (int(px),int(py)), (int(cx),int(cy)), state_color, 2)
                cv2.circle(frame, (int(px),int(py)), 6, state_color, -1)

            # status text
            text = f"STATE: {state}"
            cv2.putText(frame, text, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, state_color, 2)
            if min_dist is not None:
                cv2.putText(frame, f"dist(px): {int(min_dist)}", (10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (230,230,230), 2)

            # danger overlay
            if state == "DANGER":
                
                t = int(time.time() * 2)  # change 2 -> frequency
                if t % 2 == 0:
                    cv2.putText(frame, "DANGER DANGER", (w//6, h//4), cv2.FONT_HERSHEY_DUPLEX, 2.2, (0,0,255), 5)
                    
                    overlay = frame.copy()
                    cv2.rectangle(overlay, (0,0), (w,h), (0,0,255), -1)
                    alpha = 0.12
                    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

            # FPS calc
            now = time.time()
            dt = now - last_time
            last_time = now
            if dt > 0:
                fps = 0.9 * fps + 0.1 * (1.0/dt) if fps>0 else (1.0/dt)
            cv2.putText(frame, f"FPS: {fps:.1f}", (w-150, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200,200,200), 2)

            
            if show_mask:
                
                mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
                combined = np.hstack((cv2.resize(frame, (int(w*2/3), int(h*2/3))),
                                      cv2.resize(mask_bgr, (int(w*2/3), int(h*2/3)))))
                cv2.imshow("Frame + Mask", combined)
            else:
                cv2.imshow("Hand Boundary POC", frame)


        k = cv2.waitKey(1) & 0xFF
        if k == ord('q'):
            break
        elif k == ord('s'):
            show_mask = not show_mask
        elif k == ord('p'):
            paused = not paused

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
